In [1]:
import os
import math
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import datasets
from datasets import load_dataset

C:\Users\gguaquiere\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [3]:
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# **Data**

In [5]:
def load_train_test_val_data(name_dataset : str):

    data = load_dataset('silicone',name_dataset)
    train = pd.DataFrame(data['train'])
    test = pd.DataFrame(data['test'])
    val = pd.DataFrame(data['validation'])

    return train, test, val

In [ ]:
swda_train, swda_test, swda_val = load_train_test_val_data("swda")

Reusing dataset silicone (C:\Users\gguaquiere\.cache\huggingface\datasets\silicone\swda\1.0.0\af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)
100%|██████████| 3/3 [00:00<00:00, 42.93it/s]


In [ ]:
swda_train.Dialogue_Act.value_counts()

In [ ]:
swda_train = swda_train[swda_train.Dialogue_Act.isin(['sd',
                                                               'b',
                                                               'sv_fx',
                                                               '+',
                                                               '%'])]

swda_test = swda_test[swda_test.Dialogue_Act.isin(['sd',
                                                               'b',
                                                               'sv_fx',
                                                               '+',
                                                               '%'])]

In [ ]:
train = swda_train[['Utterance','Dialogue_Act']]
test = swda_test[['Utterance','Dialogue_Act']]
val = swda_val[['Utterance','Dialogue_Act']]

In [ ]:
os.makedirs("model", exist_ok=True)
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir = 'model/uncased_L-12_H-768_A-12'
bert_ckpt_file = 'model/uncased_L-12_H-768_A-12/bert_model.ckpt'
bert_config_file = 'model/uncased_L-12_H-768_A-12/bert_config.json'

# **Input Text Preparation**

In [ ]:
class DataPreparation:
    
    text_column = "Utterance"
    label_column = "Dialogue_Act"

    def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes

        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self.prepare_data, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self.data_padding, [self.train_x, self.test_x])

    def prepare_data(self, df):
        x, y = [], []

        for _, row in tqdm(df.iterrows()):
            text, label = row[DataPreparation.text_column], row[DataPreparation.label_column]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))

        return np.array(x), np.array(y)

    def data_padding(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)

In [ ]:
tokenizer = FullTokenizer(vocab_file=bert_ckpt_dir+"/vocab.txt")

# **Model**

In [ ]:
def model_defination(max_seq_len, bert_ckpt_file):
    
    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
        
    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)

    return model

In [ ]:
classes = train.Dialogue_Act.unique().tolist()

data = DataPreparation(train, test, tokenizer, classes, max_seq_len=128)

In [ ]:
model = model_defination(data.max_seq_len, bert_ckpt_file)

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=3,
  #callbacks=[tensorboard_callback]
)